In [1]:
import pandas as pd
import numpy as np
import altair as alt

# Comparing the winning rate of different nation 

in All men's doubles (MD) matches played during the BWF World Tour.

## 1. merge data

In [2]:
file_name = ["md"]

final_data = pd.DataFrame()
for f in file_name:
    d = pd.read_csv("data/" + f + ".csv")
    if "team_one_nationalities" not in d.columns:
        d["team_one_nationalities"] = d["team_one_player_one_nationality"]
        d["team_two_nationalities"] = d["team_two_player_one_nationality"]
    final_data = pd.concat([final_data, d])

In [3]:
final_data.to_csv("data/final_data.csv")

## 2. operate data

In [4]:
data = pd.read_csv("data/final_data.csv")
data.sample(5)

,Unnamed: 0,tournament,city,country,date,tournament_type,discipline,round,winner,nb_sets,...,team_one_game_points_game_2,team_two_game_points_game_2,game_2_scores,team_one_most_consecutive_points_game_3,team_two_most_consecutive_points_game_3,team_one_game_points_game_3,team_two_game_points_game_3,game_3_scores,team_one_nationalities,team_two_nationalities
805,805,VICTOR Korea Open 2018,Seoul,Korea,26-09-2018,HSBC BWF World Tour Super 500,MD,Round of 32,2,3,...,0.0,2.0,"['0-0', '1-0', '1-1', '1-2', '1-3', '1-4', '1-...",2.0,8.0,0.0,1.0,"['0-0', '1-0', '1-1', '2-1', '3-1', '3-2', '3-...",DEN,TPE
1216,1216,PRINCESS SIRIVANNAVARI Thailand Masters 2019,Bangkok,Thailand,09-01-2019,HSBC BWF World Tour Super 300,MD,Round of 32,2,2,...,0.0,1.0,"['0-0', '1-0', '1-1', '2-1', '2-2', '2-3', '2-...",NaN,NaN,NaN,NaN,[],IND,THA
2002,2002,YONEX-SUNRISE Vietnam Open 2019,Ho Chi Minh City,Vietnam,11-09-2019,BWF Tour Super 100,MD,Round of 32,1,3,...,2.0,0.0,"['0-0', '1-0', '2-0', '2-1', '3-1', '4-1', '4-...",6.0,3.0,1.0,0.0,"['0-0', '1-0', '2-0', '3-0', '4-0', '5-0', '6-...",INA,JPN
1886,1886,IDBI Federal Life Insurance Hyderabad Open 2019,Hyderabad,India,07-08-2019,BWF Tour Super 100,MD,Round of 32,2,3,...,5.0,0.0,"['0-0', '0-1', '0-2', '1-2', '2-2', '2-3', '2-...",3.0,5.0,0.0,1.0,"['0-0', '1-0', '2-0', '2-1', '2-2', '2-3', '2-...",MAS,IND
68,68,PERODUA Malaysia Masters 2018,Bukit Jalil,Malaysia,20-01-2018,HSBC BWF World Tour Super 500,MD,Semi final,2,2,...,0.0,3.0,"['0-0', '0-1', '0-2', '1-2', '1-3', '1-4', '2-...",NaN,NaN,NaN,NaN,[],TPE,MAS


In [5]:
len(data)

2804

In [6]:
diff_nat_data = data[data['team_one_nationalities'] != data["team_two_nationalities"]]
winner_nat = []
loser_nat = []
for i in range(len(diff_nat_data)):
    if data.iloc[i]['winner'] == 1:
        winner_nat.append(data.iloc[i]["team_one_nationalities"])
        loser_nat.append(data.iloc[i]["team_two_nationalities"])
    else:
        winner_nat.append(data.iloc[i]["team_two_nationalities"])
        loser_nat.append(data.iloc[i]["team_one_nationalities"])
diff_nat_data['winner_nationalities'] = winner_nat
diff_nat_data['loser_nationalities'] = loser_nat

/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/2529909722.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_nat_data['winner_nationalities'] = winner_nat
/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/2529909722.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_nat_data['loser_nationalities'] = loser_nat


### 2.1 get nation data

In [7]:
nation_data = pd.DataFrame()
nat_winning_rate = {}
for i in range(len(diff_nat_data)):
    cur_winning_nat = diff_nat_data.iloc[i]['winner_nationalities']
    cur_losing_nat = diff_nat_data.iloc[i]['loser_nationalities']
    if cur_winning_nat not in nat_winning_rate.keys():
        nat_winning_rate[cur_winning_nat] = [1, 0]
    else:
        nat_winning_rate[cur_winning_nat][0] += 1
    cur_data = pd.DataFrame({'nation':[cur_winning_nat],'date': [diff_nat_data.iloc[i]['date']], 'winning_matches':[nat_winning_rate[cur_winning_nat][0]], 'losing_matches':[nat_winning_rate[cur_winning_nat][1]]})
    nation_data = pd.concat([nation_data,cur_data])
    
    if cur_losing_nat not in nat_winning_rate.keys():
        nat_winning_rate[cur_losing_nat] = [0, 1]
    else:
        nat_winning_rate[cur_losing_nat][1] += 1 
    cur_data = pd.DataFrame({'nation':[cur_losing_nat],'date': [diff_nat_data.iloc[i]['date']], 'winning_matches':[nat_winning_rate[cur_losing_nat][0]], 'losing_matches':[nat_winning_rate[cur_losing_nat][1]]})
    nation_data = pd.concat([nation_data,cur_data])

In [8]:
nation_data = nation_data.reset_index().drop(["index"], axis = 1)
nation_data["total_matches"] = nation_data["winning_matches"] + nation_data["losing_matches"]
nation_data["winning_rate"] = nation_data["winning_matches"] / nation_data["total_matches"]
nation_data["final_day"] = np.zeros(len(nation_data))

In [9]:
nat = []
nat_total_cnt = len(nation_data["nation"].unique())
for i in range(len(nation_data) - 1, 0, -1):
    if len(nat) == nat_total_cnt:
        break
    if nation_data.iloc[i]["nation"] not in nat:
        nat.append(nation_data.iloc[i]["nation"])
        nation_data.loc[i, "final_day"] = 1

In [10]:
nation_data['date'] = pd.to_datetime(nation_data['date'])
nation_data

/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/271137754.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  nation_data['date'] = pd.to_datetime(nation_data['date'])


,nation,date,winning_matches,losing_matches,total_matches,winning_rate,final_day
0,HKG,2018-09-01,1,0,1,1.000000,0.0
1,THA,2018-09-01,0,1,1,0.000000,0.0
2,INA,2018-09-01,1,0,1,1.000000,0.0
3,THA,2018-09-01,0,2,2,0.000000,0.0
4,THA,2018-09-01,1,2,3,0.333333,0.0
...,...,...,...,...,...,...,...
4935,DEN,2021-03-27,139,124,263,0.528517,1.0
4936,CHN,2021-03-27,337,240,577,0.584055,0.0
4937,CHN,2021-03-27,337,241,578,0.583045,1.0
4938,MAS,2021-03-28,227,221,448,0.506696,1.0


## 3. plot

In [22]:
#change color when selection
selection=alt.selection_interval(empty="none")

colorCondition=alt.condition(selection, "nation", alt.value("lightgray"))

#scatter plot
scatter_chart = alt.Chart(nation_data).mark_circle(filled = True, size = 60).transform_filter(
    alt.datum.final_day == 1
).encode(
    x=alt.X('total_matches:Q', title = "final total matches"),
    y=alt.Y('winning_rate:Q', title = "Winning rate of the nation"),
    color = alt.value("lightgray")
)

# line chart
line_chart = alt.Chart(nation_data).mark_line().encode(
    alt.Y('winning_rate:Q', title = "Winning rate of the nation"),
    alt.X('date:T'),
    alt.Color('nation:N')
)

# add selection to each component
scatter_chart = scatter_chart.add_selection(
    selection
).encode(
    color=colorCondition,
)

line_chart = line_chart.transform_filter(selection)


#Put all together
(scatter_chart & line_chart).configure_view(strokeWidth=0).properties(
    title = "The winning rate of different nation in All men's doubles (MD) matches played during the BWF World Tour."
)


alt.VConcatChart(...)